In [ ]:
# Fill missing values
data.fillna(data.median(), inplace=True)

# Feature engineering
data['transaction_hour'] = data['transaction_date'].dt.hour
data['transaction_day_of_week'] = data['transaction_date'].dt.dayofweek

# Aggregating customer-level features
customer_features = data.groupby('customer_id').agg({
    'transaction_amount': ['mean', 'sum', 'count'],
    'fraud': ['sum', 'mean']
}).reset_index()

customer_features.columns = ['customer_id', 'customer_avg_transaction_amount', 'customer_total_transaction_amount', 'customer_transaction_count', 'customer_fraud_count', 'customer_fraud_rate']

data = pd.merge(data, customer_features, on='customer_id', how='left')

data = pd.get_dummies(data, columns=['merchant', 'transaction_category'], drop_first=True)

# Dropping unnecessary columns
data.drop(['transaction_date'], axis=1, inplace=True)

# Feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data[['transaction_amount', 'customer_avg_transaction_amount', 'customer_total_transaction_amount']] = scaler.fit_transform(data[['transaction_amount', 'customer_avg_transaction_amount', 'customer_total_transaction_amount']])

X = data.drop(['fraud'], axis=1)
y = data['fraud']
